In [23]:
%matplotlib inline 
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import ipywidgets as widgets
from ipywidgets import interact
from pathlib import Path
import copy

import os
import torch

import typhon_model


In [41]:
root_path = Path('results')
bootstrap_exp_name = '20230513_bootstrap_7'

model_path = (root_path / bootstrap_exp_name / 'models' / 'bootstrap_model').with_suffix('.pth')
print(model_path)

# Load model
model_dict = torch.load(model_path, map_location=torch.device('cpu'))
model = typhon_model.TyphonModel.from_state_dict(model_dict)

results/20230513_bootstrap_7/models/bootstrap_model.pth


In [42]:
new_model = copy.deepcopy(model)

### Remove BUS_SELECTED module

In [43]:
new_model.dms.pop('BUS_SELECTED')
print()

### Change name to BUSI --> UDIAT

In [44]:
new_model.dms.update({'UDIAT': new_model.dms['BUSI']})
new_model.dms.pop('BUSI')
print()

### Update names

In [45]:
new_model.dsets_names = [k for k in new_model.dms.keys()]

### Check

In [46]:
new_model.dms.keys()

odict_keys(['LINK_BRATS_LGG_flair', 'BRAIN', 'UDIAT'])

In [47]:
new_model.dsets_names

['LINK_BRATS_LGG_flair', 'BRAIN', 'UDIAT']

## Save new model

In [48]:
OUTPUT_FOLDER = root_path / 'bootstrap_models_adapted' / bootstrap_exp_name 
assert not os.path.exists(OUTPUT_FOLDER)
os.mkdir(OUTPUT_FOLDER)
torch.save(new_model.to_state_dict(), OUTPUT_FOLDER / 'bootstrap_model.pth')